# 二創 Scenario

## 1. Initialize agent

In [ ]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 2. DID exchange 

### Build DM-SP connection

In [ ]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("#######################")
print("Invitation - Copy this!!")
print(invite_message)

### > Break here - switch to SP to receive invitation <

In [ ]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

In [ ]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

In [ ]:
#####################
# get SP connection #
#####################

connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
sp_connection = connection["connection_id"]
print(sp_connection)

------

### Build DM-MUSIC connection

In [ ]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("#######################")
print("Invitation - Copy this!!")
print(invite_message)

### > Break here - switch to MUSIC to receive invitation <

In [ ]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

In [ ]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

In [ ]:
#####################
# get MUSIC connection #
#####################

connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
music_connection = connection["connection_id"]
print(music_connection)

### **Build listener for DID exchange**

In [ ]:
my_did = await agent_controller.wallet.get_public_did()
my_did = my_did['result']['did']
print("my DID", my_did)
sp_did = 'None'
music_did = 'None'

def messages_handler(payload):
    global sp_did
    global music_did
    connection_id = payload["connection_id"]
    did = payload["content"]
    if did[:3] == 'sp_':
        sp_did = did[3:]
        print('sp_did', sp_did)
    else:
        music_did = did[6:]
        print('music_did', music_did)
        
    asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(connection_id, my_did))
    print("Handle message", payload, connection_id)
    
def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
    
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}
message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener, connection_listener], defaults=False)

In [ ]:
print(music_did)
print(sp_did)

## 4. Give SP the VC of re-create right - DM
Scenario:     
When SP pay for the re-create right, the SP can show DM the certificate.     
On receiving the certificate, the DM will give SP the VC, so that the Music platform will recognize the VC and allow it to put VC on the platform


### SP (re-creater) re-creates the song, this is the info of the song it recreates on (These information is fetched from 3. in SP)

In [ ]:
target_song = "F4zojhjnUqDAAzYsdpqmtQ"
origin = "PQRXDxdGqQGSZ8z69p4xZP"

### Write Ownership

In [ ]:
# Define you schema name - must be unique on the ledger
schema_name = "recreate_schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["origin", "target", "time", "owner"]

# Write schema to ledger
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

# Write cred. def.
response = await agent_controller.definitions.write_cred_def(schema_id)
cred_def_id = response["credential_definition_id"]
print(cred_def_id)




### Write VC

In [ ]:
# Get nym 
nym = await agent_controller.wallet.get_public_did()
print(nym["result"]["did"])

# get schema id
created_schema = await agent_controller.schema.get_created_schema(schema_issuer_did='PQRXDxdGqQGSZ8z69p4xZP')
print(created_schema)
schema_id = created_schema["schema_ids"][0]

In [ ]:
# get cred_def_id
definitions = await agent_controller.definitions.search_created(schema_id=schema_id)
print(definitions["credential_definition_ids"])
cred_def_id = definitions["credential_definition_ids"][0]

In [ ]:
response = await agent_controller.connections.get_connections()
results = response['results']
print(len(results))
# print("Results : ", results)
if len(results) > 0:
    connection = response['results']
    for con in connection:
        print("Connection :", con)
        if con['state'] == 'active' and con["their_label"] == "Carol":        
            connection_id = con["connection_id"]
            print("Active Connection ID : ", connection_id)
        else: print("Inactive Connection ID: ", con["connection_id"], "with state: ", con["state"])
else:
    print("You must create a connection")

In [ ]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "target", "value": target_song},
    {"name": "time", "value": time},
    {"name": "origin", "value": origin},
    {"name": "owner", "value": sp_did},
]
print(credential_attributes)


# send credential
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

### > Break: Go to SP for credential exchange <

### Get records

In [ ]:
response = await agent_controller.issuer.get_records()
print(response['results'])
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

In [ ]:
## Remove records
response = await agent_controller.issuer.remove_record(record_id)
print(response)

# Terminate Agent

In [ ]:
response = await agent_controller.terminate()
print(response)